<a href="https://colab.research.google.com/github/karlmanalo/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [3]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.

In [4]:
# Checking object type of df['created']

df['created']

0        2016-06-24 07:54:24
1        2016-06-12 12:19:27
2        2016-04-17 03:26:41
3        2016-04-18 02:22:02
4        2016-04-28 01:32:41
                ...         
49347    2016-06-02 05:41:05
49348    2016-04-04 18:22:34
49349    2016-04-16 02:13:40
49350    2016-04-08 02:13:33
49351    2016-04-12 02:48:07
Name: created, Length: 48817, dtype: object

In [5]:
# Converting df['created'] to datetime format to filter by month

df['created'] = pd.to_datetime(df['created'])
df['created']

0       2016-06-24 07:54:24
1       2016-06-12 12:19:27
2       2016-04-17 03:26:41
3       2016-04-18 02:22:02
4       2016-04-28 01:32:41
                ...        
49347   2016-06-02 05:41:05
49348   2016-04-04 18:22:34
49349   2016-04-16 02:13:40
49350   2016-04-08 02:13:33
49351   2016-04-12 02:48:07
Name: created, Length: 48817, dtype: datetime64[ns]

In [6]:
# Creating training dataset

train = df[(df['created'].dt.month == 4) | (df['created'].dt.month == 5)]
train.shape

(31844, 34)

In [7]:
# Creating test dataset

test = df[df['created'].dt.month == 6]
test.shape

(16973, 34)

In [8]:
# Making sure all data is captured in training and test datasets. 
# Should come out to 48817

test.shape[0] + train.shape[0]

48817

- [ ] Engineer at least two new features. (See below for explanation & ideas.)

In [0]:
# New column: Number of perks. This totally did not take hours.

df['number_of_perks'] = df.iloc[:,10:34].sum(axis=1)

In [10]:
# Checking how many descriptions have NaN values.

df['description'].isnull().sum()

1425

In [11]:
# The above is incorrect as we can see that the following code is empty but
# returns a string of whitespace.

df.iloc[1][3]

'        '

In [0]:
# Replacing descriptions that just have whitespace with NaN values.
# Need help understanding r'^\s*$' and regex. I assume r'^\s*$' is calling
# anything with only 2 or more spaces in the description but I am lost. Also not
# sure what regex=True does.

df['description'] = df['description'].replace(r'^\s*$', np.NaN, regex=True)

In [13]:
# This result looks more right.

df['description'].isnull().sum()

3288

In [0]:
# Defining new column: Has Description = 1, Doesn't Have Description = 0

df['has_description'] = (df['description'].isnull().map({False:1, True:0}))

In [0]:
# New column: Total rooms

df['total_rooms'] = df['bedrooms'] + df['bathrooms']

In [0]:
# New column: Bedrooms:Bathrooms

df['beds_to_baths'] = df['bedrooms']/df['bathrooms']

In [0]:
# New column: Total rooms:Bathrooms

df['total_to_baths'] = df['total_rooms']/df['bathrooms']

In [0]:
# New column: Has doorman and elevator

df['doorman_and_elev'] = ((df['doorman'] == 1) & (df['elevator'] == 1)).map({True:1, False:0})

In [0]:
# New column: Has fitness center and laundry in unit

df['fitness_and_laundry'] = ((df['laundry_in_unit'] == 1) & (df['fitness_center'] == 1)).map({True:1, False:0})

In [20]:
df.corr()

,bathrooms,bedrooms,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,number_of_perks,has_description,total_rooms,beds_to_baths,total_to_baths,doorman_and_elev,fitness_and_laundry
bathrooms,1.000000,0.524082,0.013407,-0.020099,0.687296,0.132882,0.021475,0.096922,0.024539,0.157173,0.172230,0.129547,-0.013202,0.148334,-0.016214,0.209366,0.102113,0.144450,0.233038,0.089073,0.136990,0.113315,0.066826,0.140767,-0.001944,0.014260,0.096283,0.066940,-0.003403,0.238312,0.034814,0.744468,0.012748,0.012748,0.152751,0.167990
bedrooms,0.524082,1.000000,0.003490,0.055117,0.535503,-0.024821,-0.011173,0.096420,-0.009750,-0.047562,0.152102,0.163000,0.008558,0.013150,0.001420,0.151790,0.045451,0.124912,0.189415,0.061006,0.097772,0.033441,0.000907,0.099149,-0.014880,-0.107341,0.070760,0.012306,0.003234,0.116616,0.033854,0.958785,0.836614,0.836614,-0.020584,0.063736
latitude,0.013407,0.003490,1.000000,0.329185,-0.036286,-0.010523,-0.029808,0.018653,-0.030954,-0.043393,-0.020612,-0.026788,-0.041635,-0.108455,0.026802,-0.044339,-0.063198,-0.079919,0.017146,-0.030969,0.016877,0.027020,-0.056093,0.006466,-0.060054,-0.015650,-0.000589,-0.068330,-0.117199,-0.068345,0.013817,0.007209,-0.005065,-0.005065,-0.039949,-0.047756
longitude,-0.020099,0.055117,0.329185,1.000000,-0.251004,-0.189836,-0.058475,-0.108493,-0.070329,-0.275734,-0.169220,-0.088033,-0.044562,-0.254960,0.000196,-0.130139,-0.161466,-0.098595,-0.024793,-0.125635,-0.035474,-0.075046,-0.108001,-0.049016,0.046755,-0.060018,-0.029846,-0.063635,-0.102955,-0.256118,-0.012327,0.036503,0.078773,0.078773,-0.261776,-0.138611
price,0.687296,0.535503,-0.036286,-0.251004,1.000000,0.207169,0.051453,0.101503,0.060401,0.276215,0.223899,0.132240,-0.019417,0.228775,-0.029122,0.271195,0.122929,0.142146,0.242911,0.090269,0.139140,0.134513,0.071431,0.145973,-0.013251,0.007100,0.103672,0.072517,0.011517,0.305263,-0.018633,0.649097,0.214368,0.214368,0.251276,0.206214
elevator,0.132882,-0.024821,-0.010523,-0.189836,0.207169,1.000000,0.033347,0.270831,0.034833,0.614558,0.349832,0.227895,0.141097,0.431070,-0.097015,0.134158,0.332028,0.204343,0.200591,0.277666,0.168081,0.183664,0.184178,0.135329,0.025895,0.054918,0.084056,0.155396,0.114882,0.597606,0.065084,0.024873,-0.109778,-0.109778,0.736533,0.237310
cats_allowed,0.021475,-0.011173,-0.029808,-0.058475,0.051453,0.033347,1.000000,-0.177633,0.937245,0.088480,-0.047880,-0.024052,0.105644,0.126886,0.047500,-0.008827,0.027590,0.066713,-0.022404,0.077759,0.019979,0.009704,0.042710,0.006152,0.031414,-0.037966,0.006690,0.039656,0.104496,0.317865,-0.074516,-0.001594,-0.021306,-0.021306,0.072423,0.023136
hardwood_floors,0.096922,0.096420,0.018653,-0.108493,0.101503,0.270831,-0.177633,1.000000,-0.185663,0.205119,0.634983,0.342971,-0.144728,0.167510,0.013435,0.360716,0.278727,0.188913,0.317281,0.237935,0.178089,0.170589,0.187635,0.181005,-0.194436,0.116572,0.161751,0.124829,-0.120489,0.503269,0.242575,0.107922,0.046711,0.046711,0.256266,0.241861
dogs_allowed,0.024539,-0.009750,-0.030954,-0.070329,0.060401,0.034833,0.937245,-0.185663,1.000000,0.095434,-0.043839,-0.011414,0.093035,0.131521,0.051610,0.002510,0.034104,0.067859,-0.016154,0.089425,0.024472,0.009675,0.055655,0.005219,0.032729,-0.041014,0.009229,0.048123,0.106071,0.325645,-0.073010,0.000543,-0.020416,-0.020416,0.073736,0.030649
doorman,0.157173,-0.047562,-0.043393,-0.275734,0.276215,0.614558,0.088480,0.205119,0.095434,1.000000,0.312710,0.257031,0.077216,0.604863,-0.054614,0.166397,0.388921,0.205656,0.195814,0.312036,0.160789,0.263833,0.219943,0.126936,-0.074747,0.010080,0.075956,0.168161,0.126708,0.636769,0.075051,0.015150,-0.147155,-0.147155,0.900647,0.313855


- [ ] Fit a linear regression model with at least two features.

In [0]:
# Re-instantiating train/test datasets after creating new features

train = df[(df['created'].dt.month == 4) | (df['created'].dt.month == 5)]
test = df[df['created'].dt.month == 6]


In [0]:
from sklearn.linear_model import LinearRegression

In [0]:
model = LinearRegression()

In [0]:
features = ['bathrooms', 'bedrooms', 'doorman', 'laundry_in_unit']
target = ['price']

In [0]:
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [26]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [27]:
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error: ${mae:.2f}')

Train Error: $761.92


In [28]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error: ${mae:.2f}')

Test Error: $772.48


- [ ] Get the model's coefficients and intercept.


In [29]:
print('Intercept', model.intercept_[0])
coefficients = pd.Series(model.coef_[0], features)
print(coefficients.to_string())

Intercept 334.73930794813487
bathrooms          1823.683492
bedrooms            434.394449
doorman             690.340646
laundry_in_unit     480.306697


- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.

In [0]:
# Importing regression metrics, defining function to return regression metrics.

from sklearn.metrics import mean_squared_error, r2_score

def squared_errors(df, features, target):
  mse = mean_squared_error(df[target], y_pred)
  rmse = np.sqrt(mse)
  mae = mean_absolute_error(df[target], y_pred)
  r2 = r2_score(df[target], y_pred)
  print('Mean Squared Error:', mse)
  print('Root Mean Squared Error:', rmse)
  print('Mean Absolute Error:', mae)
  print('R^2:', r2)

In [0]:
# Re-instantiating y_pred as model predicting X_train

y_pred = model.predict(X_train)

In [47]:
squared_errors(train, features, target)

Mean Squared Error: 1358454.0861448655
Root Mean Squared Error: 1165.5273854117995
Mean Absolute Error: 761.9182489584003
R^2: 0.5624986342096916


In [0]:
# Re-instantiating y_pred as model predicting X_test

y_pred = model.predict(X_test)

In [49]:
squared_errors(test, features, target)

Mean Squared Error: 1340423.8295163002
Root Mean Squared Error: 1157.7667422742372
Mean Absolute Error: 772.4828326946163
R^2: 0.5687211527859187


- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!

Tried a lot of stuff above.